In [1]:
import xlrd
import csv
import pickle
import numpy as np
import copy
from matplotlib import pyplot as plt
import math
import random
import pandas as pd
from numpy.random import choice
from scipy import stats
from collections import defaultdict
from operator import itemgetter, sub
from sklearn.metrics import roc_curve, auc, accuracy_score, explained_variance_score, \
    mean_squared_error, mean_absolute_error, median_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Lasso
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
class FileToSamples:
    'convert the input .csv file to samples(Dataframe):\
    [Iteration,CellId,PilotPower,Load,Users,TBS,NB1Id*5,NB1Power*5,NB1Users*5,NB1Load*5\
    AveragePower,AverageLoad,AverageUser,TimeAveragePower,TimeAverageLoad,TimeAverageUser] '
    
    def __init__(self,filename):
        self.samples = pd.read_pickle(filename) # type DataFrame (pandas package)  
        # use for test feature selection
        self.feature = FileToSamples.__getFeatues(self.samples)  # type DataFrame (pandas package) 
        self.target=self.samples['TBS']
    @staticmethod
    def __getFeatues(samples):
        Testnumber =9
        feature=[]
        #nnEstimater = [TrainingNNModel()]*Testnumber
        feature=[pd.DataFrame()]*Testnumber

        feature[0]= samples[['PilotPower']]
        feature[1]= samples[['PilotPower','TimeAverageUsers','TimeAverageLoad']]
        feature[2]= samples[['PilotPower','Load','Users']]
        feature[3]= samples[['PilotPower','Load','Users','TimeAverageUsers','TimeAverageLoad']]
        feature[4]= samples[['PilotPower','NB1Power','NB2Power','NB3Power','NB4Power','NB5Power']]
        feature[5]= samples[['PilotPower','Load','Users','NB1Power','NB2Power','NB3Power','NB4Power','NB5Power']]
        feature[6]= samples[['PilotPower','Load','Users','TimeAverageUsers','TimeAverageLoad','NB1Power','NB2Power','NB3Power','NB4Power','NB5Power','AverageLoad','AverageUser']]
        feature[7]= samples[['PilotPower','NB1Power','NB2Power','NB3Power','NB4Power','NB5Power','TimeAverageUsers','TimeAverageLoad']]
        feature[8]= samples[['PilotPower','Load','Users','NB1Power','NB2Power','NB3Power','NB4Power','NB5Power','AverageLoad','AverageUser']]
        return feature




In [3]:
class TrainingNNModel:
    'to train neural network for the features'
    def __init__(self):
        self.model = MLPRegressor(hidden_layer_sizes=(20,), activation='logistic', alpha = 0.0001,\
                                  max_iter = 20000, tol=1e-4) # neural network model
        self.scaler = StandardScaler() # scaler of input
        
    def train(self,feature,target):
        self.scaler.fit(feature)
        self.model.fit(self.scaler.transform(feature),target)
        self.score= self.model.score(self.scaler.transform(feature),target)
        self.estimateTarget = self.model.predict(self.scaler.transform(feature))
        
    def test(self,Testfeature,TestTarget):
        self.TestScore= self.model.score(self.scaler.transform(Testfeature),TestTarget)
        self.TestEstimateTarget = self.model.predict(self.scaler.transform(Testfeature))

In [4]:
class result:
        def __init__(self,trainData,testData):
            
            Testnumber=9
            #train
            self.Estimater=[]
            for i in range(Testnumber):
                p= TrainingNNModel()
                p.train(trainData.feature[i],trainData.target)
                self.Estimater.append(p)

            for i in range(Testnumber):
                self.Estimater[i].test(testData.feature[i],testData.target) 

            #record
            ScoreN11=[]
            for i in range(Testnumber):
                ScoreN11.append(self.Estimater[i].score)                 
            self.score11=pd.DataFrame.from_dict(pd.Series(ScoreN11,name='R^2'))
           
            self.TBS11= trainData.samples[['TBS']]
            for i in range(Testnumber):
                self.TBS11.loc[:,'Estimate%d'%(i)]=self.Estimater[i].estimateTarget

            ScoreN12=[]
            for i in range(Testnumber):
                ScoreN12.append(self.Estimater[i].TestScore)
            self.score12=pd.DataFrame.from_dict(pd.Series(ScoreN12,name='R^2'))

            self.TBS12= testData.samples[['TBS']]
            for i in range(Testnumber):
                self.TBS12.loc[:,'Estimate%d'%(i)]=self.Estimater[i].TestEstimateTarget




In [5]:
if __name__ == '__main__':
    trainingFile = 'Data__Random3.csv.pkl'
    #trainingFile = 'Workbook2.csv'
    testFile = 'Data__Random4.csv.pkl'
    
    
    testData = FileToSamples(testFile)
    trainData = FileToSamples(trainingFile)

#display setting
#pd.set_option('max_columns', 40)
#pd.set_option('max_rows', 200)
#data.samples

In [6]:
result1=result(trainData,testData)
result2=result(testData,trainData)

/Users/Fuwei/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Fuwei/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:561: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [7]:
resultall = pd.concat([result1.score11, result1.score12,result2.score11,result2.score12], axis=1, join='inner')
resultall

,R^2,R^2,R^2,R^2
0,0.038551,0.035365,0.035058,0.036061
1,0.723473,0.713774,0.710053,0.716113
2,0.746424,0.635559,0.691896,0.669376
3,0.856449,0.798035,0.836670,0.829062
4,0.072408,0.068234,0.073448,0.066116
5,0.739031,0.727961,0.719541,0.724004
6,0.866869,0.804136,0.830689,0.828364
7,0.881760,0.824162,0.847793,0.841298
8,0.755254,0.747146,0.746176,0.746020


In [39]:
mse=9*[None]

In [41]:
for i in range(9):
    mse[i] = metrics.mean_squared_error(testData.target,result1.TBS12.iloc[:,i+1])    

In [42]:
mse

[16532.850156686258,
 4905.623030508963,
 6246.149748623001,
 3461.4678588055808,
 15969.516912996143,
 4662.4732639440817,
 3356.9109582797951,
 3013.6846585904755,
 4333.6526918807876]